## Berechnungen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabeichung für die aufgenommenen Messwerte    
der Android App zu berechnen. Zu einem Bild werden parallel Messwerte erfasst von dem Beschleunigungssensor und Magnetsensor   
des Smartphones, innerhalb eines Zeitfenster mit der festen Länge von 80ms. Bei dem Beschleunigungssensor sind dies die Messwerte   
der 3-Achsen X,Y sowie Z. Desweiteren werden die Gier-Nick-Roll Winkel in rad in dem Zeitfenster erfasst.       
Es wird in dem Notebook auch die Gieränderung in rad berechnet. Die Ergebnisse können in die Merkmal CSV Datei geschrieben werden.

In [1]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [2]:
featuresDf = pandas.read_csv("../daten/merkmaleRoh.csv")
featuresDf.describe()

,Zeitstempel in Unixzeit,Breitengrad,Laengengrad,Geschwindigkeit in km/h,Hoehe in m ueber WGS 84,Anzahl der Messwerte,Start des Zeitfensters in ns seit Start der JVM,Start der Messwerterfassung in ns seit Start der JVM,Start der Belichtung in ns seit Start der JVM,Belichtungszeit in ns,Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM,Stopp der Messwerterfassung in Unixzeit
count,1.935000e+03,1935.000000,1935.000000,1935.000000,1935.000000,1935.000000,1.935000e+03,1.935000e+03,1.935000e+03,1.935000e+03,1.935000e+03,1.935000e+03
mean,1.521815e+12,53.611756,10.141385,15.431479,70.983613,19.335401,1.232928e+14,1.232927e+14,1.232928e+14,1.717044e+07,1.232930e+14,1.521815e+12
std,1.711999e+05,0.001963,0.004292,2.578698,2.540434,0.574053,1.712002e+11,1.712002e+11,1.712005e+11,5.915285e+06,1.711998e+11,1.711998e+05
min,1.521815e+12,53.608086,10.136243,6.012000,64.299440,14.000000,1.229923e+14,1.229923e+14,1.229924e+14,1.000184e+07,1.229925e+14,1.521815e+12
25%,1.521815e+12,53.610050,10.137212,14.256000,69.145140,19.000000,1.231447e+14,1.231447e+14,1.231448e+14,1.000184e+07,1.231449e+14,1.521815e+12
50%,1.521815e+12,53.612152,10.140454,15.768001,71.069275,19.000000,1.232957e+14,1.232956e+14,1.232957e+14,2.000368e+07,1.232958e+14,1.521815e+12
75%,1.521815e+12,53.613777,10.145028,17.208002,72.942380,20.000000,1.234378e+14,1.234377e+14,1.234378e+14,2.000368e+07,1.234379e+14,1.521815e+12
max,1.521815e+12,53.613907,10.149695,20.268002,77.023680,22.000000,1.235899e+14,1.235898e+14,1.235899e+14,2.999632e+07,1.235900e+14,1.521815e+12


In [3]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1935 entries, 0 to 1934
Data columns (total 19 columns):
Zeitstempel in Unixzeit                                               1935 non-null int64
Breitengrad                                                           1935 non-null float64
Laengengrad                                                           1935 non-null float64
Geschwindigkeit in km/h                                               1935 non-null float64
Hoehe in m ueber WGS 84                                               1935 non-null float64
X-Achse Beschleunigungswerte in m/s^2                                 1935 non-null object
Y-Achse Beschleunigungswerte in m/s^2                                 1935 non-null object
Z-Achse Beschleunigungswerte in m/s^2                                 1935 non-null object
Gier Messwerte in rad                                                 1935 non-null object
Nick Messwerte in rad                                                 1

In [4]:
# Zeigt die ersten 5 Reihen Beschleunigungssensordaten der X-Achse 
featuresDf['X-Achse Beschleunigungswerte in m/s^2'].head()

0    -0.10847 -0.42383 0.22795 -0.24662 0.32374 0.6...
1    -0.14057 -0.58741 -0.46987 -0.32986 -0.11061 -...
2    -0.52854 -0.101 -1.53641 -0.6468 0.402 -0.5517...
3    1.51295 0.56685 0.26967 1.44163 1.41385 -0.140...
4    2.09385 2.38 0.44838 0.20553 1.75806 -0.24836 ...
Name: X-Achse Beschleunigungswerte in m/s^2, dtype: object

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
X-Achse Beschleunigungswerte in m/s^2   
Y-Achse Beschleunigungswerte in m/s^2   
Z-Achse Beschleunigungswerte in m/s^2   
Gier Messwerte in rad    
Nick Messwerte in rad          
Roll Messwerte in rad                   
Zeitstempel der Messwerte in ns               
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [5]:
accXList = featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()

In [6]:
# Direkt mit dem Numpy Array ist es nicht möglich die Konvertierung durchzuführen, weil die Arrays von Typ object sind.   
pandas.DataFrame(featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ")).values

array([[list(['-0.10847', '-0.42383', '0.22795', '-0.24662', '0.32374', '0.65745', '1.23087', '1.03073', '0.71738', '0.2675', '0.13745', '0.32455', '0.33631', '0.02394', '-0.42516', '-0.432', '-0.20764', '0.14041', '0.08175'])],
       [list(['-0.14057', '-0.58741', '-0.46987', '-0.32986', '-0.11061', '-0.08843', '-0.37714', '-1.03715', '-0.93692', '-0.27447', '-0.25015', '-1.10412', '-1.22034', '-1.0375', '-0.78397', '0.5221', '1.91939', '0.15651', '-0.48765'])],
       [list(['-0.52854', '-0.101', '-1.53641', '-0.6468', '0.402', '-0.55176', '-0.85505', '-0.00978', '-0.23761', '-0.69568', '1.3129', '1.98509', '0.39294', '-0.69691', '0.14739', '-0.31107', '0.25686', '0.25152', '-0.13582'])],
       ...,
       [list(['0.26091', '-1.21625', '-0.16082', '1.09724', '0.63268', '0.19975', '0.3284', '0.12488', '0.19191', '0.12293', '0.28228', '0.28718', '0.56691', '0.11648', '0.0626', '0.14207', '0.08308', '0.03587', '0.42716'])],
       [list(['0.13186', '0.33538', '-0.09938', '-0.29396', '

In [7]:
accXList[1] # zeige Liste

['-0.14057',
 '-0.58741',
 '-0.46987',
 '-0.32986',
 '-0.11061',
 '-0.08843',
 '-0.37714',
 '-1.03715',
 '-0.93692',
 '-0.27447',
 '-0.25015',
 '-1.10412',
 '-1.22034',
 '-1.0375',
 '-0.78397',
 '0.5221',
 '1.91939',
 '0.15651',
 '-0.48765']

In [8]:
np.array(accXList[1]).dtype

dtype('<U8')

In [9]:
# Konvertiert die Python Objekte in der Liste zu den Type Float.
# Das Ergebnis wird in einem Dataframe geschrieben
accList = []
accList.append(pandas.DataFrame(np.array(accXList[0]).astype(float)).transpose())
for x in accXList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())

In [10]:
accXDf = pandas.concat(accList, ignore_index=True)

In [11]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.10847,-0.42383,0.22795,-0.24662,0.32374,0.65745,1.23087,1.03073,0.71738,0.26750,...,0.33631,0.02394,-0.42516,-0.43200,-0.20764,0.14041,0.08175,NaN,NaN,NaN
1,-0.14057,-0.58741,-0.46987,-0.32986,-0.11061,-0.08843,-0.37714,-1.03715,-0.93692,-0.27447,...,-1.22034,-1.03750,-0.78397,0.52210,1.91939,0.15651,-0.48765,NaN,NaN,NaN
2,-0.52854,-0.10100,-1.53641,-0.64680,0.40200,-0.55176,-0.85505,-0.00978,-0.23761,-0.69568,...,0.39294,-0.69691,0.14739,-0.31107,0.25686,0.25152,-0.13582,NaN,NaN,NaN
3,1.51295,0.56685,0.26967,1.44163,1.41385,-0.14065,-0.28102,0.66397,0.19414,0.00213,...,-1.00578,-0.88118,-0.56698,-0.16238,-0.35969,0.55507,1.85382,NaN,NaN,NaN
4,2.09385,2.38000,0.44838,0.20553,1.75806,-0.24836,1.57883,2.50428,5.57371,5.40905,...,-4.09078,-6.10730,-4.60997,-1.00641,-0.63652,-2.56242,-0.70146,NaN,NaN,NaN


In [12]:
# Um mehr Spalten eines Panda DataFrames auszugeben
pandas.set_option('display.max_columns', 40)
accXDf.abs().T.mean().head(20)

0     0.386513
1     0.622851
2     0.581849
3     0.653394
4     2.425357
5     0.712010
6     0.387993
7     0.616517
8     0.367582
9     0.317555
10    0.676903
11    0.280649
12    0.259970
13    0.363707
14    0.362819
15    0.358905
16    0.333961
17    0.451120
18    0.416946
19    0.526957
dtype: float64

In [13]:
accXDf.shape

(1935, 22)

In [14]:
# Die Anzahl der Messerte muss immer gleich sein, ansonsten werden fehlende Messwerte
# mit NaN gefüllt.
featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 22]

Series([], Name: Anzahl der Messwerte, dtype: int64)

In [15]:
# ausführen wenn alle Daten die gleiche Ahnzahl an Messwerten haben sollen. 
# Dadurch werden Reihen gelöscht!
unwantedRows = featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] != 20]
featuresDf.drop(featuresDf.index, axis=0)

,Zeitstempel in Unixzeit,Breitengrad,Laengengrad,Geschwindigkeit in km/h,Hoehe in m ueber WGS 84,X-Achse Beschleunigungswerte in m/s^2,Y-Achse Beschleunigungswerte in m/s^2,Z-Achse Beschleunigungswerte in m/s^2,Gier Messwerte in rad,Nick Messwerte in rad,Roll Messwerte in rad,Zeitstempel der Messwerte in ns,Anzahl der Messwerte,Start des Zeitfensters in ns seit Start der JVM,Start der Messwerterfassung in ns seit Start der JVM,Start der Belichtung in ns seit Start der JVM,Belichtungszeit in ns,Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM,Stopp der Messwerterfassung in Unixzeit


In [16]:
accYList = featuresDf['Y-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(accYList[0]).astype(float)).transpose())
for x in accYList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accYDf = pandas.concat(accList, ignore_index=True)

In [17]:
accYDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1935 entries, 0 to 1934
Data columns (total 22 columns):
0     1935 non-null float64
1     1935 non-null float64
2     1935 non-null float64
3     1935 non-null float64
4     1935 non-null float64
5     1935 non-null float64
6     1935 non-null float64
7     1935 non-null float64
8     1935 non-null float64
9     1935 non-null float64
10    1935 non-null float64
11    1935 non-null float64
12    1935 non-null float64
13    1935 non-null float64
14    1934 non-null float64
15    1934 non-null float64
16    1934 non-null float64
17    1932 non-null float64
18    1880 non-null float64
19    681 non-null float64
20    28 non-null float64
21    1 non-null float64
dtypes: float64(22)
memory usage: 332.7 KB


In [18]:
accYDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,-1.39422,-1.39126,-1.23567,-1.08055,-1.07903,-0.17378,1.07141,0.81108,-0.14800,0.60170,0.78774,0.38495,-0.87199,-0.75896,-0.37740,-0.86894,-0.64926,-0.24368,0.14209,NaN,NaN,NaN
1,0.46738,0.20528,0.04156,-0.16602,0.48002,0.85894,0.88628,0.93878,-0.76601,-2.16050,-2.60188,-1.14689,0.84455,1.82478,2.62429,2.83485,-0.19918,-2.33527,-2.34331,NaN,NaN,NaN
2,0.16166,2.51962,1.80110,-1.33264,-3.02752,0.13683,1.56506,-0.81662,0.86360,2.56020,3.30455,1.64758,-0.55141,-0.77830,-0.23965,0.11466,-1.47129,-0.45693,1.59571,NaN,NaN,NaN
3,-0.09097,0.54007,-0.27287,-1.03050,0.07958,1.22813,1.42680,1.23329,1.27769,1.09869,-1.78731,-2.94688,-3.26164,-2.85455,0.03004,2.35303,6.01953,4.83087,-0.62499,NaN,NaN,NaN
4,-0.81612,-0.30054,0.35709,-0.26603,2.17747,3.50403,8.70246,12.83056,6.14251,0.54691,2.24555,-0.56336,-4.84844,-2.39250,0.55290,-2.82153,-4.69364,-5.74697,-5.65493,NaN,NaN,NaN


In [19]:
# Konvertiere Beschleunigungssensordaten Z-Achse zu einem DataFrame
measuredDataList = featuresDf['Z-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accZDf = pandas.concat(accList, ignore_index=True)

In [20]:
# Konvertiere Nick Messwerte zu DataFrame
measuredDataList = featuresDf['Nick Messwerte in rad'].str.split(" ").tolist()
pitchList = []
pitchList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    pitchList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
nickDf = pandas.concat(pitchList, ignore_index=True)

In [21]:
# Konvertiere Roll Messwerte zu DataFrame
measuredDataList = featuresDf['Roll Messwerte in rad'].str.split(" ").tolist()
rollList = []
rollList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    rollList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
rollDf = pandas.concat(rollList, ignore_index=True)

In [22]:
measuredDataList = featuresDf['Gier Messwerte in rad'].str.split(" ").tolist()
azimuthList = []
azimuthList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    azimuthList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
azimuthDf = pandas.concat(azimuthList, ignore_index=True)

In [23]:
measuredDataList = featuresDf['Zeitstempel der Messwerte in ns'].str.split(" ").tolist()
timestampsList = []
timestampsList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    timestampsList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
sensorTimestampsDf = pandas.concat(timestampsList, ignore_index=True)

In [24]:
accXMean = accXDf.abs().T.mean()
accXMean.head(2)

0    0.386513
1    0.622851
dtype: float64

In [25]:
accYDf.abs().T.mean().head()

0    0.740616
1    1.248725
2    1.312891
3    1.736181
4    3.429660
dtype: float64

In [26]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'] = accXMean
featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'] = accYDf.abs().T.mean()
featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'] = accZDf.abs().T.mean()
featuresDf['Mittelwert Nick in rad'] = nickDf.abs().T.mean()
featuresDf['Mittelwert Roll in rad'] = rollDf.abs().T.mean()

In [27]:
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'].head(2)

0    0.386513
1    0.622851
Name: Mittelwert X-Achse Beschleunigung in m/s^2, dtype: float64

In [28]:
featuresDf.columns

Index(['Zeitstempel in Unixzeit', 'Breitengrad', 'Laengengrad',
       'Geschwindigkeit in km/h', 'Hoehe in m ueber WGS 84',
       'X-Achse Beschleunigungswerte in m/s^2',
       'Y-Achse Beschleunigungswerte in m/s^2',
       'Z-Achse Beschleunigungswerte in m/s^2', 'Gier Messwerte in rad',
       'Nick Messwerte in rad', 'Roll Messwerte in rad',
       'Zeitstempel der Messwerte in ns', 'Anzahl der Messwerte',
       'Start des Zeitfensters in ns seit Start der JVM',
       'Start der Messwerterfassung in ns seit Start der JVM',
       'Start der Belichtung in ns seit Start der JVM',
       'Belichtungszeit in ns',
       'Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM',
       'Stopp der Messwerterfassung in Unixzeit',
       'Mittelwert X-Achse Beschleunigung in m/s^2',
       'Mittelwert Y-Achse Beschleunigung in m/s^2',
       'Mittelwert Z-Achse Beschleunigung in m/s^2', 'Mittelwert Nick in rad',
       'Mittelwert Roll in rad'],
      dtype='object')

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [29]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, values in dfValues.iterrows():
        sum = 0
        for value in values:
            if math.isnan(value) == False:
                tempDifference = value-meansDf[i]
                sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / values.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [30]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeldifferenz in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Argument übergebenen DataFrames. Der zurückgegebene Radiant    
ist immer positiv und gibt die relative änderung in rad an.      
Prec.:   
Postc.: Radiant berechnet   

In [31]:
def calculateAngelChangeAzimuth(azimuthDf):
    result = []
    for i,values in azimuthDf.iterrows():
        first = values[0]*(180/math.pi)
        last = values.iloc[values.count()-1]*(180/math.pi)
        resultTemp = np.abs(first - last)
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append("{0:.5f}".format(round(resultTemp * ( math.pi / 180),5)))
    return np.array(result).astype(float)

In [32]:
featuresDf['GierAenderung in rad'] = calculateAngelChangeAzimuth(azimuthDf)

In [33]:
# Berechnung der Varianzen und hinzufügen der Ergebnisse als neue Spalte in dem DataFrame 
featuresDf['Varianz X-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'],accXDf)
featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'],accYDf)
featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'],accZDf)
featuresDf['Varianz Nick in rad'] = calcVariance(featuresDf['Mittelwert Nick in rad'],nickDf)
featuresDf['Varianz Roll in rad'] = calcVariance(featuresDf['Mittelwert Roll in rad'],rollDf)

In [34]:
featuresDf['Varianz X-Achse Beschleunigung in m/s^2'].head()

0     0.24693
1     1.43192
2     1.03273
3     0.76280
4    13.96020
Name: Varianz X-Achse Beschleunigung in m/s^2, dtype: float64

In [35]:
calculateStandardDeviation(featuresDf['Varianz X-Achse Beschleunigung in m/s^2'])

array([0.49692, 1.19663, 1.01623, ..., 0.44881, 0.54977, 0.47755])

In [36]:
# Berechne Standardabweichung
featuresDf['Standardabweichung X-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz X-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Y-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Nick in rad'] = calculateStandardDeviation(featuresDf['Varianz Nick in rad'])
featuresDf['Standardabweichung Roll in rad'] = calculateStandardDeviation(featuresDf['Varianz Roll in rad'])

In [37]:
featuresDf['Standardabweichung X-Achse Beschleunigung in m/s^2'].head()

0    0.49692
1    1.19663
2    1.01623
3    0.87338
4    3.73634
Name: Standardabweichung X-Achse Beschleunigung in m/s^2, dtype: float64

In [38]:
accXDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1935 entries, 0 to 1934
Data columns (total 22 columns):
0     1935 non-null float64
1     1935 non-null float64
2     1935 non-null float64
3     1935 non-null float64
4     1935 non-null float64
5     1935 non-null float64
6     1935 non-null float64
7     1935 non-null float64
8     1935 non-null float64
9     1935 non-null float64
10    1935 non-null float64
11    1935 non-null float64
12    1935 non-null float64
13    1935 non-null float64
14    1934 non-null float64
15    1934 non-null float64
16    1934 non-null float64
17    1932 non-null float64
18    1880 non-null float64
19    681 non-null float64
20    28 non-null float64
21    1 non-null float64
dtypes: float64(22)
memory usage: 332.7 KB


In [39]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,-0.10847,-0.42383,0.22795,-0.24662,0.32374,0.65745,1.23087,1.03073,0.71738,0.26750,0.13745,0.32455,0.33631,0.02394,-0.42516,-0.43200,-0.20764,0.14041,0.08175,NaN,NaN,NaN
1,-0.14057,-0.58741,-0.46987,-0.32986,-0.11061,-0.08843,-0.37714,-1.03715,-0.93692,-0.27447,-0.25015,-1.10412,-1.22034,-1.03750,-0.78397,0.52210,1.91939,0.15651,-0.48765,NaN,NaN,NaN
2,-0.52854,-0.10100,-1.53641,-0.64680,0.40200,-0.55176,-0.85505,-0.00978,-0.23761,-0.69568,1.31290,1.98509,0.39294,-0.69691,0.14739,-0.31107,0.25686,0.25152,-0.13582,NaN,NaN,NaN
3,1.51295,0.56685,0.26967,1.44163,1.41385,-0.14065,-0.28102,0.66397,0.19414,0.00213,0.49210,-0.05062,-1.00578,-0.88118,-0.56698,-0.16238,-0.35969,0.55507,1.85382,NaN,NaN,NaN
4,2.09385,2.38000,0.44838,0.20553,1.75806,-0.24836,1.57883,2.50428,5.57371,5.40905,4.12811,0.03877,-4.09078,-6.10730,-4.60997,-1.00641,-0.63652,-2.56242,-0.70146,NaN,NaN,NaN


In [40]:
# Änderungen in CSV Datei schreiben
featuresDf.set_index('Zeitstempel in Unixzeit', inplace=True)
featuresDf.to_csv('../daten/merkmale.csv')

In [41]:
pandas.read_csv("../daten/merkmale.csv").columns

Index(['Zeitstempel in Unixzeit', 'Breitengrad', 'Laengengrad',
       'Geschwindigkeit in km/h', 'Hoehe in m ueber WGS 84',
       'X-Achse Beschleunigungswerte in m/s^2',
       'Y-Achse Beschleunigungswerte in m/s^2',
       'Z-Achse Beschleunigungswerte in m/s^2', 'Gier Messwerte in rad',
       'Nick Messwerte in rad', 'Roll Messwerte in rad',
       'Zeitstempel der Messwerte in ns', 'Anzahl der Messwerte',
       'Start des Zeitfensters in ns seit Start der JVM',
       'Start der Messwerterfassung in ns seit Start der JVM',
       'Start der Belichtung in ns seit Start der JVM',
       'Belichtungszeit in ns',
       'Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM',
       'Stopp der Messwerterfassung in Unixzeit',
       'Mittelwert X-Achse Beschleunigung in m/s^2',
       'Mittelwert Y-Achse Beschleunigung in m/s^2',
       'Mittelwert Z-Achse Beschleunigung in m/s^2', 'Mittelwert Nick in rad',
       'Mittelwert Roll in rad', 'GierAenderung in rad',
     

In [42]:
# Speichert die DatenFrames mit den Messwerten in csv Dateien
accXDf.index.names = ["index"]
accXDf.to_csv('../daten/x-Achse-Beschleunigungswerte.csv')
accYDf.index.names = ["index"]
accYDf.to_csv('../daten/y-Achse-Beschleunigungswerte.csv')
accZDf.index.names = ["index"]
accZDf.to_csv('../daten/z-Achse-Beschleunigungswerte.csv')
nickDf.index.names = ["index"]
nickDf.to_csv('../daten/nick-Messwerte.csv')
rollDf.index.names = ["index"]
rollDf.to_csv('../daten/roll-Messwerte.csv')
azimuthDf.index.names = ["index"]
azimuthDf.to_csv('../daten/azimuth-Messwerte.csv')
sensorTimestampsDf.index.names = ["index"]
sensorTimestampsDf.to_csv('../daten/sensorevent-Zeitstempel.csv')

In [43]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 0.0,2)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 234.24702,5)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),0.0,2)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),451.5,1) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),21.24853,5) 
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),0.0,2)
    
    #  Testet die Funktion calculateAngelChangeAzimuth mit zwei gleichen rad Werten
    def testCalculateAngelChangeAzimuthEqual(self):
        radiants = pandas.DataFrame([[1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),0.0,2)
    
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten -1.0 und 1.0
    def testCalculateAngelChangeAzimuthOne(self):
        radiants = pandas.DataFrame([[-1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 1.0 und -1.0
    def testCalculateAngelChangeAzimuthOne2(self):
        radiants = pandas.DataFrame([[1.0,-1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  

    # Testet die Funktion calculateAngelChangeAzimuthValues mit mehreren Werten
    def testCalculateAngelChangeAzimuthValues(self):
        radiants = pandas.DataFrame([[-1.57,1.0,2.0,1.3,1.57]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),3.14,2)
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 0.05 und 0.06
    def testCalculateAngelChangeAzimuthSmall(self):
        radiants = pandas.DataFrame([[0.01,0.02]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 0.01,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde mit 
    # negative Radianten 
    def testCalculateAngelChangeAzimuthNegativ(self):
        radiants = pandas.DataFrame([[-2.9,-2.0,-1.28]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 1.62,2) 

unittest.main(argv=[''], verbosity=2, exit=False)

testCalcVariance0 (__main__.CalcTest) ... ok
testCalcVariance2 (__main__.CalcTest) ... ok
testCalcVariance5 (__main__.CalcTest) ... ok
testCalcVarianceMultidim (__main__.CalcTest) ... ok
testCalcVarianceNegative (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthEqual (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthNegativ (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne2 (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthSmall (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthValues (__main__.CalcTest) ... ok
testCalculateStandardDeviation (__main__.CalcTest) ... ok
testCalculateStandardDeviationZero (__main__.CalcTest) ... ok

----------------------------------------------------------------------
Ran 13 tests in 0.078s

OK
